In [1]:
import sys  
sys.path.insert(0, '/home/m.caros/work/objectDetection/src/model')
from pointnet import ClassificationPointNet


In [2]:
sys.path.insert(0, '/home/m.caros/work/objectDetection/utils')
from utils_plot import plot_3d_coords

In [3]:
from utils_barlow import *

In [4]:
batch_size = 128
num_workers = 20  # to run notebook on CPU
max_epochs = 20
z_dim=32

## Data

In [5]:
dataset_folder = '/dades/LIDAR/towers_detection/datasets/pc_40x40_4096p_v3'
n_points = 4096
c_sample = False

In [6]:
path_list_files='/home/m.caros/work/objectDetection/train_test_files/RGBN_40x40_barlow_p1'

# Datasets train / val / test
with open(os.path.join(path_list_files, 'train_cls_files.txt'), 'r') as f:
    train_files = f.read().splitlines()
with open(os.path.join(path_list_files, 'val_cls_files.txt'), 'r') as f:
    val_files = f.read().splitlines()
with open(os.path.join(path_list_files, 'test_cls_files.txt'), 'r') as f:
    test_files = f.read().splitlines()

# paths_files = glob.glob('/dades/LIDAR/towers_detection/datasets/pc_40x40_4096p/pylon*')
# towers_files = [f.split('/')[-1] for f in paths_files]

In [7]:
# Initialize datasets
train_dataset = BarlowTwinsDataset(dataset_folder=dataset_folder,
                             task='classification', 
                             number_of_points=n_points,
                             files=train_files,
                             fixed_num_points=True,
                             use_ground=True,
                             c_sample=c_sample)

val_dataset = BarlowTwinsDataset(dataset_folder=dataset_folder,
                           task='classification', 
                           number_of_points=n_points,
                           files=val_files,
                           fixed_num_points=True,
                           use_ground=True,
                           c_sample=c_sample)

test_dataset = BarlowTwinsDataset(dataset_folder=dataset_folder,
                           task='classification', 
                           number_of_points=n_points,
                           files=test_files,
                           fixed_num_points=True,
                           use_ground=True,
                           c_sample=c_sample)

In [8]:

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                          num_workers=num_workers, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                        num_workers=num_workers, drop_last=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                        num_workers=0, drop_last=True)



### Encoder: PointNet classifier

In [9]:
classifier = ClassificationPointNet(num_classes=2,
                                   point_dimension=3,
                                   dataset=train_dataset,
                                   device='cuda')
classifier = classifier.cuda()

encoder = classifier.base_pointnet


### Instantiate Barlow Twins model

In [10]:
encoder_out_dim=1024

ckpt_model = BarlowTwins(
    encoder=encoder,
    encoder_out_dim=encoder_out_dim,
    num_training_samples=len(train_dataset),
    batch_size=batch_size,
    z_dim=z_dim,
)

In [11]:
online_finetuner = OnlineFineTuner(encoder_output_dim=encoder_out_dim, num_classes=2)


### Load Barlow Twins trained model

In [12]:
ckpt_dict = torch.load('checkpoints_cls/epoch=16-step=6987.ckpt') 
ckpt_model.load_state_dict(ckpt_dict['state_dict'])
# encoder = ckpt_model.encoder
encoder = ckpt_model

# for batch in train_loader:
#     pc, pc_noise, labels = batch
#     gl_feat, _ = encoder(pc.cuda())
#     print(gl_feat.shape)
#     break

In [13]:
#gl_feat[0]

In [14]:
# Z
# for batch in train_loader:
#     pc, pc_noise, labels = batch
#     z = ckpt_model(pc.cuda())
#     print(z.shape)
#     break

In [15]:
cls = Classifier(num_classes=2,
                 encoder=encoder,
                 num_training_samples=len(train_dataset),
                 batch_size=batch_size,
                 max_epochs=max_epochs,
                 learning_rate=0.001, 
                 dropout=0.3).cuda()


In [16]:
checkpoint_callback = ModelCheckpoint( save_top_k=10, dirpath='checkpoints_tower_cls', monitor='barlow_cls_val_loss')

trainer = Trainer(
    max_epochs=max_epochs,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    callbacks=[checkpoint_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
torch.set_float32_matmul_precision('medium')

In [18]:
trainer.fit(cls, train_loader, val_loader)

/home/m.caros/anaconda3/envs/pytorch37/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/m.caros/work/objectDetection/src/barlow_twins/checkpoints_tower_cls exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type        | Params
------------------------------------------
0 | encoder   | BarlowTwins | 3.4 M 
1 | loss_fn   | NLLLoss     | 0     
2 | fc_1      | Linear      | 524 K 
3 | fc_2      | Linear      | 131 K 
4 | fc_3      | Linear      | 514   
5 | bn_1      | BatchNorm1d | 1.0 K 
6 | bn_2      | BatchNorm1d | 512   
7 | dropout_1 | Dropout     | 0     
------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.045    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/m.caros/anaconda3/envs/pytorch37/lib/python3.7/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## Test Classifier with Barlow Twins vector

In [19]:
# ckpt_dict = torch.load('checkpoints_tower_cls/epoch=53-step=92232.ckpt')  # upload checkpoint to aws
# cls.load_state_dict(ckpt_dict['state_dict'])
